In [ ]:
from google.colab import drive 
drive.mount ('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfdb
import keras
from tensorflow.keras.applications.vgg19 import VGG19
from keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np

A. With Pretrained model

In [ ]:
def pretrained_model (weights,input_shape):
  base_model = VGG19(weights=weights, include_top=False, input_shape=input_shape)
  return base_model

In [ ]:
DATASET_PATH = "/content/drive/MyDrive/archive (2).zip (Unzipped Files)/15-Scene/"

def prepare_data_set (DATASET_PATH):
  one_hot_lookup = np.eye(15) # 15 classes

  dataset_x_train = []
  dataset_y_train = []
  
  dataset_x_test = []
  dataset_y_test = []
  
  for category in sorted(os.listdir(DATASET_PATH)):
      s = 0
      print('loading category: '+str(int(category)))
      for fname in os.listdir(DATASET_PATH+category):
            s = s + 1 
            if (s <100):
              img = cv2.imread(DATASET_PATH+category+'/'+fname, 2)
              img = cv2.resize(img, (224,224))
              dataset_x_train.append(np.reshape(img, [224,224,1]))
              dataset_y_train.append(np.reshape(one_hot_lookup[int(category)], [15]))
            else : 
              dataset_x_test.append(np.reshape(img, [224,224,1]))
              dataset_y_test.append(np.reshape(one_hot_lookup[int(category)], [15]))

  x_train = np.array(dataset_x_train)
  y_train = np.array(dataset_y_train)
  x_test = np.array(dataset_x_test)
  y_test = np.array(dataset_y_test)

  p = np.random.permutation(len(x_train))
  x_train = x_train[p]
  y_train = y_train[p]

  p = np.random.permutation(len(x_test))
  x_test = x_test[p]
  y_test = y_test[p]
 
  return x_train , y_train , x_test , y_test



In [ ]:
import cv2
X_train , Y_train , X_test , Y_test = prepare_data_set (DATASET_PATH)

loading category: 0
loading category: 1
loading category: 2
loading category: 3
loading category: 4
loading category: 5
loading category: 6
loading category: 7
loading category: 8
loading category: 9
loading category: 10
loading category: 11
loading category: 12
loading category: 13
loading category: 14


In [ ]:
def label_encoder (labels) : 
  encoded_labels = [list(labels[i]).index(1) for i in range (labels.shape[0]) ]
  return encoded_labels

In [ ]:
Y_train = label_encoder (Y_train)
Y_test = label_encoder (Y_test)

In [ ]:
base_model_1 = pretrained_model ('imagenet',input_shape=(224,224, 3))

80150528/80134624 [==============================] - 1s 0us/step


In [ ]:
def feature_extractor_1(X) : 
  rgb_batch = np.repeat(X[..., np.newaxis], 3, -2)
  rgb_batch = rgb_batch[:,:,:,:,0]
  extracted_features = base_model_1.predict(rgb_batch)
  exctracted_features = np.reshape(extracted_features,(X.shape[0],7*7*512) )
  return exctracted_features

In [ ]:
import time
start = time.time()
train_exctracted_features = feature_extractor_1(X_train)
print("Total time: ", time.time() - start, "seconds")


Total time:  61.68860173225403 seconds


In [ ]:
from sklearn.svm import SVC
import time
start = time.time()
svm_model_linear = SVC(kernel = 'rbf', C = 10).fit(train_exctracted_features,Y_train )
print("Total time: ", time.time() - start, "seconds")


Total time:  170.03956842422485 seconds


In [ ]:
import time
start = time.time()
test_exctracted_features = feature_extractor_1(X_test)
print("Total time: ", time.time() - start, "seconds")


Total time:  6.694871664047241 seconds


In [ ]:
svm_predictions = svm_model_linear.predict(test_exctracted_features)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score (svm_predictions,Y_test)


0.8023575638506876

B . No pretrained model

In [ ]:
from keras import layers
base_model_2 = keras.Sequential(
    [
        keras.Input(shape=(224,224,5)),
        layers.Conv2D(32, 3, padding="valid", activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, activation="relu"),
        layers.MaxPooling2D(),
        # layers.Conv2D(256, 3, activation="relu"),
        # layers.MaxPooling2D(),
        layers.Conv2D(512, 4, activation="relu"),
    ])

In [ ]:
base_model_2.output

<KerasTensor: shape=(None, 23, 23, 512) dtype=float32 (created by layer 'conv2d_66')>

In [ ]:
def feature_extractor_2(X) : 
  rgb_batch = np.repeat(X[..., np.newaxis], 5, -2)
  rgb_batch = rgb_batch[:,:,:,:,0]
  extracted_features = base_model_2.predict(rgb_batch)
  exctracted_features = np.reshape(extracted_features,(X.shape[0],23*23*512) )
  return exctracted_features

In [ ]:
import time
start = time.time()
train_exctracted_features = feature_extractor_2(X_train)
print("Total time: ", time.time() - start, "seconds")


Total time:  5.250510931015015 seconds


In [ ]:
from sklearn.svm import SVC
import time 
start = time.time()
svm_model_linear = SVC(kernel = 'rbf', C = 10).fit(train_exctracted_features,Y_train )
print("Total time: ", time.time() - start, "seconds")


In [ ]:
import time 
start = time.time()
test_exctracted_features = feature_extractor_2(X_test)
print("Total time: ", time.time() - start, "seconds")


In [ ]:
svm_predictions = svm_model_linear.predict(test_exctracted_features)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score (svm_predictions,Y_test)